### Exploration

In [1]:
!git clone https://github.com/openai/baselines.git
%cd baselines
!pip install -e .
!pip install stable-baselines
!pip install --upgrade tensorflow==1.14.0
!pip install --upgrade tensorflow-gpu==1.14.0
!pip install gym
!pip install pyarrow
!pip uninstall --yes tensorboard-plugin-wit
%cd ..

fatal: destination path 'baselines' already exists and is not an empty directory.
/project/to-run-for-results/nsl=kdd/baselines
Obtaining file:///project/to-run-for-results/nsl%3Dkdd/baselines
     |████████████████████████████████| 1.6 MB 14.8 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 45.1 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.15.7-py3-none-any.whl size=1648839 sha256=3d0ea4b75b8c5dbc6acac2b8f83b4b858f0ba32d86b7cccd4344aff3fedd14f9
  Stored in directory: /home/rppeeter/.cache/pip/wheels/be/72/05/d3dfcfc2a31bbf886112b6373881bdf2e9e00d2c943f3b4f91
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=cfe7756d6ded18d768de55b44040531326f28987f4a698b3e1e99a4bae9fe7e9
  Stored in directory: /home/rppeeter/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0

In [2]:
import os
import gym
import time
import json
import random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env.dummy_vec_env import DummyVecEnv

from stable_baselines import A2C
from stable_baselines import bench
from stable_baselines import logger
import tensorflow as tf

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2719213667706452704,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 16153623178727706818
 physical_device_desc: "device: XLA_CPU device"]

In [3]:
nslkdd_train = pd.read_feather("/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTrain.feather")
nslkdd_test = pd.read_feather("/project/datasets/clean-ids-collection/nsl-kdd/clean/KDDTest.feather")
# data = pd.concat([nslkdd_test], ignore_index=True)
# print(len(data.columns))

nslkdd_train.protocol_type = pd.Categorical(nslkdd_train.protocol_type).codes
nslkdd_train.service = pd.Categorical(nslkdd_train.service).codes
nslkdd_train.flag = pd.Categorical(nslkdd_train.flag).codes


nslkdd_test.protocol_type = pd.Categorical(nslkdd_test.protocol_type).codes
nslkdd_test.service = pd.Categorical(nslkdd_test.service).codes
nslkdd_test.flag = pd.Categorical(nslkdd_test.flag).codes


# print(pd.get_dummies(nslkdd_train,columns=['protocol_type' ,'service', 'flag']))
# nslkdd_test = pd.get_dummies(nslkdd_test)
nslkdd_train.head(5)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [4]:
nslkdd_train.info()
nslkdd_train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  int8   
 2   service                      125973 non-null  int8   
 3   flag                         125973 non-null  int8   
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,125973.00000,125973.000000,125973.000000,125973.000000,1.259730e+05,1.259730e+05,125973.000000,125973.000000,125973.000000,125973.000000,...,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000,125973.000000
mean,287.14465,1.053202,31.226469,6.979996,4.556674e+04,1.977911e+04,0.000198,0.022687,0.000111,0.204409,...,182.148945,115.653005,0.521242,0.082951,0.148379,0.032542,0.284452,0.278485,0.118832,0.120240
std,2604.51531,0.426620,16.346470,2.689365,5.870331e+06,4.021269e+06,0.014086,0.253530,0.014366,2.149968,...,99.206213,110.702741,0.448949,0.188922,0.308997,0.112564,0.444784,0.445669,0.306557,0.319459
min,0.00000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,1.000000,20.000000,5.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,82.000000,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,1.000000,24.000000,9.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,255.000000,63.000000,0.510000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.00000,1.000000,49.000000,9.000000,2.760000e+02,5.160000e+02,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000
max,42908.00000,2.000000,69.000000,10.000000,1.379964e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Preprocessing done by Laurens D'Hooge

## Creating the model
- Build an agent to classify network flow automatically
- Feed a packet that gets classified
- Want the classification to be equal to the label

In [5]:
def balancing_dataset_sampler_df(df, train_frac=0.2, val_frac=0.1, test_frac=0.7):
    col = df.columns[-1]
    print(col)
    cols = df.columns[:-1]
    print(cols)
    vc = df[col].value_counts()
    print(vc)
    n = vc.iloc[-1]
    print(n)
    m = vc.iloc[0]
    print(m)
    print(int(m-n))
    initial_cut = df.loc[df[col] == vc.index[0]].sample(n=int(m-n), replace=False)
    print(initial_cut.index)
    df = df.drop(index=initial_cut.index)
    vc = df[col].value_counts()
    print(vc)
    print(int(n*train_frac))
    train_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*train_frac), replace=False))
    train_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=train_df.index)

    validation_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*val_frac), replace=False))
    validation_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=validation_df.index)

    test_df = df.groupby(col).apply(lambda x: x.sample(n=int(n*test_frac), replace=False))
    test_df.reset_index(level=0, inplace=True, drop=True)
    df = df.drop(index=test_df.index)

    return train_df[cols], train_df[col], validation_df[cols], validation_df[col], test_df[cols], test_df[col]


In [6]:
def nslkdd_split_df(train_df, test_df):
    train_col = train_df.columns[-1]
    print(train_col)
    train_cols = train_df.columns[:-1]
    print(train_cols)
    test_col = test_df.columns[-1]
    print(test_col)
    test_cols = test_df.columns[:-1]
    print(test_cols)

    return train_df[train_cols], train_df[train_col], test_df[test_cols], test_df[test_col]

In [7]:
print(nslkdd_train['class'].value_counts())
nslkdd_train['class'] = nslkdd_train['class'].astype('object')
atk_idx = nslkdd_train.loc[nslkdd_train['class'] != "normal"].index
nslkdd_train.loc[atk_idx, 'class'] = 1.0
nslkdd_train.loc[nslkdd_train.index.difference(atk_idx), 'class'] = 0.0
nslkdd_train['class'] = nslkdd_train['class'].astype(dtype=np.float32)

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: class, dtype: int64


In [8]:
print(nslkdd_test['class'].value_counts())
nslkdd_test['class'] = nslkdd_test['class'].astype('object')
atk_idx = nslkdd_test.loc[nslkdd_test['class'] != "normal"].index
nslkdd_test.loc[atk_idx, 'class'] = 1.0
nslkdd_test.loc[nslkdd_test.index.difference(atk_idx), 'class'] = 0.0
nslkdd_test['class'] = nslkdd_test['class'].astype(dtype=np.float32)

normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
loadmodule            2
udpstorm              2
perl                  2
worm                  2
sqlattack             2
phf                   2
imap                  1
Name: class, dtype: int64


In [9]:
x_train, y_train, x_test, y_test  = nslkdd_split_df(nslkdd_train, nslkdd_test)

class
Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate'],
      dtype='object')
class
Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_fai

In [10]:
y_train = y_train.to_frame()
y_test = y_test.to_frame()

# custom keys -> replace by index

x_train = x_train.set_index([pd.Index(range (0, len(x_train)))])
y_train = y_train.set_index([pd.Index(range (0, len(y_train)))])
x_test = x_test.set_index([pd.Index(range (0, len(x_test)))])
y_test = y_test.set_index([pd.Index(range (0, len(y_test)))])


### Custom environment

In [11]:
class IdsEnv(gym.Env):
    def __init__(self, images_per_episode=1, dataset=(x_train, y_train), random=True):
        # Actions we can take, classify as malicious or non-malicious (later also the correct attack)
        self.action_space = gym.spaces.Discrete(2)
         # All the features we have, len(important_features) - 1 features and 1 label. Label should not be included
        self.observation_space = gym.spaces.Box(low=float('-inf'), high=float('inf'), shape=(len(nslkdd_train.columns) - 1,))
        self.images_per_episode = images_per_episode
        self.step_count = 0

        self.x, self.y = dataset
        self.random = random
        self.dataset_idx = 0
    
    def step(self, action):
        done = False
        reward = int(action == self.expected_action)
        current_label = self.expected_action
        obs = self._next_obs()

        self.step_count += 1
        if self.step_count >= self.images_per_episode:
            done = True

        return obs, reward, done, {'label': current_label}

    def _next_obs(self):
        if self.random:
            next_obs_idx = random.randint(0, len(self.x) - 1)
            self.expected_action = int(self.y.iloc[next_obs_idx,:])
            obs = self.x.iloc[next_obs_idx,:]

        else:
            obs = self.x.iloc[self.dataset_idx]
            self.expected_action = int(self.y.iloc[self.dataset_idx])

            self.dataset_idx += 1
            if self.dataset_idx >= len(self.x):
                raise StopIteration()
        return obs
    
    def reset(self):
        self.step_count = 0

        obs = self._next_obs()
        return obs

Train classifier using dqn

In [12]:
def ids_ddqn():
    env = IdsEnv(images_per_episode=1)
    env = bench.Monitor(env, logger.get_dir())

    model = A2C(
        MlpPolicy, 
        env, 
        verbose=1,
    )
    model.learn(
        total_timesteps=int(1.0e6),
        log_interval=10000,
        
    )

    env.close()
    
    return model

start_time = time.time()
ddqn_model_1 = ids_ddqn()
print()
print("A2C 1 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_2 = ids_ddqn()
print()
print("A2C 2 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_3 = ids_ddqn()
print()
print("A2C 3 Training Time:", time.time() - start_time)
start_time = time.time()
ddqn_model_4 = ids_ddqn()
print()
print("A2C 4 Training Time:", time.time() - start_time)


Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.4      |
| explained_variance | -0.675   |
| fps                | 13       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 0.589    |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.94     |
| explained_variance | 0.531    |
| fps                | 338      |
| nupdates           | 10000    |
| policy_entropy     | 0.0537   |
| total_timesteps    | 50000    |
| value_loss         | 0.0972   |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.94     |
| e

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.6      |
| explained_variance | -1.11    |
| fps                | 17       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 1.43     |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.96     |
| explained_variance | -0.0976  |
| fps                | 340      |
| nupdates           | 10000    |
| policy_entropy     | 0.0858   |
| total_timesteps    | 50000    |
| value_loss         | 0.213    |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.93     |
| explained_variance | nan      |
| fps                | 341      |
| nupdates           | 20000    |
| policy_entropy     | 0.000148 |
| total_timesteps    | 100000   |
| value_loss         | 0.00117  |
--------------

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.4      |
| explained_variance | -0.15    |
| fps                | 16       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 0.28     |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.94     |
| explained_variance | nan      |
| fps                | 339      |
| nupdates           | 10000    |
| policy_entropy     | 0.108    |
| total_timesteps    | 50000    |
| value_loss         | 0.000838 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.97     |
| explained_variance | nan      |
| fps                | 339      |
| nupdates           | 20000    |
| policy_entropy     | 0.124    |
| total_timesteps    | 100000   |
| value_loss         | 0.000379 |
--------------

/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0        |
| explained_variance | nan      |
| fps                | 16       |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 0.423    |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.96     |
| explained_variance | nan      |
| fps                | 337      |
| nupdates           | 10000    |
| policy_entropy     | 0.167    |
| total_timesteps    | 50000    |
| value_loss         | 0.000537 |
---------------------------------
---------------------------------
| ep_len_mean        | 1        |
| ep_reward_mean     | 0.96     |
| explained_variance | nan      |
| fps                | 337      |
| nupdates           | 20000    |
| policy_entropy     | 0.0187   |
| total_timesteps    | 100000   |
| value_loss         | 7.56e-06 |
--------------

In [13]:
# 0 is benign (positive), 1 is malicious (negative) 
def ids_eval(model):
    TP, FP, TN, FN = 0,0,0,0
    env = IdsEnv(images_per_episode=1, dataset=(x_test, y_test), random=False)
    obs, done = env.reset(), False
    try:
        while True:
            obs, done = env.reset(), False
            while not done:
                obs, rew, done, info = env.step(model.predict(obs)[0])
                label = info['label']
                if label == 0 and rew > 0:
                    TP += 1
                if label == 0 and rew == 0:
                    FP += 1
                if label == 1 and rew > 0:
                    TN += 1
                if label == 1 and rew == 0:
                    FN += 1

    except StopIteration:
        accuracy = (float(TP + TN) / (TP + FP + FN + TN)) 
        precision = (float(TP) / (TP + FP))
        recall = (float(TP) / (TP + FN)) # = TPR = Sensitivity
        FPR = (float(FP) / (TN + FP)) # 1 - specificity
        f1_score = 2 * (precision * recall) / (precision + recall)
        print()
        print('validation done...')
        print('Accuracy: {0}%'.format(accuracy * 100))
        print('Precision: {0}%'.format(precision * 100))
        print('Recall/TPR/Sensitivity: {0}%'.format(recall * 100))
        print('FPR: {0}%'.format(FPR * 100))
        print('F1 score: {0}'.format(f1_score))
    return [accuracy, precision, recall, FPR, f1_score]

models = [ddqn_model_1, ddqn_model_2, ddqn_model_3, ddqn_model_4]

results_1 = ids_eval(ddqn_model_1)
results_2 = ids_eval(ddqn_model_2)
results_3 = ids_eval(ddqn_model_3)
results_4 = ids_eval(ddqn_model_4)
total_results = [-1,-1,-1,-1,-1]
accuracies = [results_1[0], results_2[0], results_3[0], results_4[0]]

for i in range(len(results_1)):
    total_results[i] = (results_1[i] + results_2[i] + results_3[i] + results_4[i] )/ 4

print()    
print('Total validation done...')
print('Accuracy: {0}%'.format(total_results[0] * 100))
print('Precision: {0}%'.format(total_results[1] * 100))
print('Recall/TPR/Sensitivity: {0}%'.format(total_results[2] * 100))
print('FPR: {0}%'.format(total_results[3] * 100))
print('F1 score: {0}'.format(total_results[4]))

highest = 0
for i in range(4):
    if accuracies[i] > accuracies[highest]:
        highest = i

print()
print("Saving model {0}".format(highest + 1))
models[highest].save('a2c_cicddos2019_allfeat.pkl')



/opt/conda/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))



validation done...
Accuracy: 74.74048442906575%
Precision: 91.27544661404238%
Recall/TPR/Sensitivity: 64.41870693446708%
FPR: 9.438202247191011%
F1 score: 0.7553072625698324

validation done...
Accuracy: 76.42622659923698%
Precision: 96.84254258412962%
Recall/TPR/Sensitivity: 65.04813729593972%
FPR: 3.7037037037037033%
F1 score: 0.7782322009848928

validation done...
Accuracy: 75.68095111347706%
Precision: 94.47444952222683%
Recall/TPR/Sensitivity: 64.75865014950875%
FPR: 6.261770244821093%
F1 score: 0.7684379488045958

validation done...
Accuracy: 76.16005678289415%
Precision: 92.16867469879519%
Recall/TPR/Sensitivity: 65.76256113828369%
FPR: 8.333333333333332%
F1 score: 0.7675806591125336

Total validation done...
Accuracy: 75.75192973116849%
Precision: 93.69027835479851%
Recall/TPR/Sensitivity: 64.99701387954981%
FPR: 6.934252382262285%
F1 score: 0.7673895178679636

Saving model 2
